# Exploratory Data Analysis - MIMIC-IV Dataset in PostgreSQL

The notebook has been implemented using Python 3.10.11.  
The MIMIC-IV v2.2 dataset has been loaded into PostgreSQL server running PostgreSQL 15.2 (Ubuntu 15.2-1.pgdg22.04+1).  
We suggest creating a virtual environment for this notebook.  
You need to install the following packages to run this notebook:

| Package Name | License                                                                                                                 | Documentation                           |
|--------------|-------------------------------------------------------------------------------------------------------------------------|-----------------------------------------|
| psycopg2     | [![License: LGPL v3](https://img.shields.io/badge/License-LGPL_v3-blue.svg)](https://www.gnu.org/licenses/lgpl-3.0)     | [Docs](https://www.psycopg.org/)        |
| pandas       | [![License](https://img.shields.io/badge/License-BSD_3--Clause-blue.svg)](https://opensource.org/licenses/BSD-3-Clause) | [Docs](https://pandas.pydata.org/)      |
| numpy        | [![License](https://img.shields.io/badge/License-BSD_3--Clause-blue.svg)](https://opensource.org/licenses/BSD-3-Clause) | [Docs](https://numpy.org/)              |
| seaborn      | [![License](https://img.shields.io/badge/License-BSD_3--Clause-blue.svg)](https://opensource.org/licenses/BSD-3-Clause) | [Docs](https://seaborn.pydata.org/)     |
| scipy        | [![License](https://img.shields.io/badge/License-BSD_3--Clause-blue.svg)](https://opensource.org/licenses/BSD-3-Clause) | [Docs](https://scipy.org/)              |
| tomli        | [![License: MIT](https://img.shields.io/badge/License-MIT-blue.svg)](https://opensource.org/licenses/MIT)               | [Docs](https://github.com/hukkin/tomli) |
| tqdm         | [![License](https://img.shields.io/pypi/l/tqdm.svg)](https://github.com/tqdm/tqdm/blob/master/LICENCE)                  | [Docs](https://tqdm.github.io/)         |
| matplotlib   | [(BSD-compatible, PSF-based)](https://matplotlib.org/stable/users/project/license.html)                                 | [Docs](https://matplotlib.org/)         |


In [ ]:
"""Update pip and install requirements."""
%pip install --upgrade pip
%pip install -r requirements.txt

In [ ]:
"""Relevant imports for EDA; setup and styling."""

# data manipulation
import numpy as np
import pandas as pd

# data vizualisation
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.gridspec import GridSpec

# tqdm for progress bars
from tqdm import tqdm

# default styling for plots
plt.style.use("ggplot")  # gnuplot style
rcParams["figure.figsize"] = 12, 6  # figure size
from matplotlib.colors import ListedColormap

# hls colormap for sns styled pie charts using matplotlib
hls = ListedColormap(sns.color_palette("hls").as_hex())

In [ ]:
"""Functions for database connection, query execution, dataframe plotting."""

import tomli as toml
import psycopg2 as pg
from typing import Any


def read_config(path: str) -> dict:
    """Read config file and return config dict."""
    with open(path, "rb") as f:
        config = toml.load(f)["database"]
    return config


def connect_to_db(config: dict) -> Any:
    """Connect to database and return connection object."""
    conn = pg.connect(**config)
    cur = conn.cursor()
    return conn, cur


def read_sql(path: str) -> str:
    """Read SQL file and returns string"""
    with open(path, "r") as f:
        sql = f.read()
    return sql


def sql_to_df(path: str, params: dict = None) -> pd.DataFrame:
    """Read SQL file, execute query and return pandas DataFrame.
    
    Optionally, pass parameters to query using the params dict.
    """
    if params is None:
        params = {}
    conn, cur = connect_to_db(read_config("./config.toml"))
    cur.execute(read_sql(path), params)
    df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
    conn.close()
    return df


def plot_corr_matrix(
    df: pd.DataFrame,
    method: str = "pearson",
    title: str="",
    figsize=(10, 5),
    linewidth=0.3,
    fmt=".2f",
    annot_kws={"size": 10},
    cmap="Spectral_r",
    cbar=True,
    ax=None,
    cbar_kws={"shrink": 0.8},
) -> None:
    """Plot heatmap of correlation matrix."""
    # set figure size
    if ax is None:
        plt.subplots(figsize=figsize)
    corr = df.corr(method)
    sns.heatmap(
        corr,
        cbar=cbar,  # show color bar? yes/no
        annot=True,  # show numbers in cells? yes/no
        square=True,  # square cells? yes/no
        linewidths=linewidth,  # linewidth between cells
        fmt=fmt,  # precision
        annot_kws=annot_kws,  # size of numbers in cells
        yticklabels=df.columns,  # y-axis labels
        xticklabels=df.columns,  # x-axis labels
        cmap=cmap,  # color palette
        ax=ax,  # axes object
        cbar_kws=cbar_kws,  # shrink color bar
    )
    if title:
        plt.title(title)
    if ax is None:
        plt.show()


def plot_boxplot_grid(df: pd.DataFrame, target: str) -> None:
    """Plot boxplots of multiple columns against a single target variable."""
    # calculate number of rows and columns
    n_cols = int(np.ceil(np.sqrt(len(df.columns) - 1)))
    n_rows = int(np.ceil((len(df.columns) - 1) / n_cols))
    # create figure and axes
    fig, axes = plt.subplots(
        nrows=n_rows, ncols=n_cols, figsize=(n_cols * 6, n_rows * 5)
    )
    # iterate over columns, rows and create boxplots
    for col, ax in zip(df.columns.drop(target), axes.flatten()):
        sns.boxplot(x=target, y=col, data=df, ax=ax)
        # set title to column name vs. target
        ax.set_title(f"{col} vs. {target}")
    plt.show()


def plot_corr_matrix_diff(
    df_one: pd.DataFrame,
    df_two: pd.DataFrame,
    method: str = "pearson",
    figsize=(10, 5),
    cmap="vlag",
    title="",
    ax=None,
) -> None:
    """Plot heatmap of difference of correlation matrices."""
    # calculate difference of correlation matrices
    corr_diff = df_one.corr(method) - df_two.corr(method)
    # plot heatmap
    plt.subplots(figsize=figsize)
    # draw arrows in cells according to correlation difference?
    sns.heatmap(
        corr_diff,
        annot=True,
        annot_kws={"size": 10},
        cbar=True,
        cmap=cmap,
        fmt=".2f",
        square=True,
        center=0,
        ax=ax,
    )
    plt.title(title)
    if ax is None:
        plt.show()


def plot_pie_chart(df, col="race", title="", ax=None, cmap=hls, explode=.1):
    """Plot pie chart for a given column in a dataframe."""
    explode = [explode] * len(df[col].value_counts())
    df[col].value_counts().plot.pie(
        shadow=True,
        autopct="%1.1f%%",
        startangle=90,
        title=title,
        cmap=cmap,
        ax=ax,
        labeldistance=1.1,
        pctdistance=0.5,
        explode=explode,
    )
    if ax is None:
        plt.show()

### Patient Age Distribution - General Hospital Population

In [ ]:
gen_pop_age = sql_to_df("./sql/demographics_age_patients.sql")

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(18, 6))
fig.suptitle("Age Distribution of Hospital Patients", fontsize=16)
# create color palette
palette = dict(zip(["F", "M"], sns.color_palette()))
# plot histograms
a1 = sns.histplot(
    data=gen_pop_age,
    x="age",
    hue="gender",
    kde=True,
    multiple="layer",
    ax=axes[0],
    palette=palette,
)
a2 = sns.histplot(
    data=gen_pop_age,
    x="age",
    hue="gender",
    multiple="fill",
    ax=axes[1],
    palette=palette,
    stat="percent",
)
# set title, x- and y-labels for subplots
a1.set(xlabel="Age", ylabel="Count", title="Age Distribution")
a2.set(xlabel="Age", ylabel="Percent", title="Gender Distribution")
# add labels to bars
# for container in a1.containers:
# a1.bar_label(container)

### Patient Age Distribution - ICU Population

In [ ]:
icu_pop_age = sql_to_df("./sql/demographics_age_icu.sql")

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(18, 6), sharex=True)
# set title
fig.suptitle("Age Distribution of Patients in ICU", fontsize=16)
# create color palette
palette = dict(zip(["F", "M"], sns.color_palette()))
# plot histograms
a1 = sns.histplot(
    data=icu_pop_age.copy(),
    x="age",
    hue="gender",
    kde=True,
    multiple="layer",
    ax=axes[0],
    palette=palette,
)
a2 = sns.histplot(
    data=icu_pop_age.copy(),
    x="age",
    hue="gender",
    multiple="fill",
    ax=axes[1],
    palette=palette,
    stat="percent",
)
# set title, x- and y-labels for subplots
a1.set(xlabel="Age", ylabel="Count", title="Age Distribution")
a2.set(xlabel="Age", ylabel="Percent", title="Gender Distribution")

### Age Distribution of Patients with Sepsis-3

In [ ]:
sepsis_pop_age = sql_to_df("./sql/demographics_age_sepsis.sql")

In [ ]:
# create color palette
palette = dict(zip(["F", "M"], sns.color_palette()))
# plot histograms
g = sns.displot(
    sepsis_pop_age,
    x="age",
    col="sepsis",
    hue="gender",
    facet_kws=dict(margin_titles=True),
    binwidth=2,
    height=6,
    kde=True,
    palette=palette,
)
# set title
g.set_titles(col_template="Sepsis={col_name}")
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle("Age Distribution of Patients with and without Sepsis", fontsize=16)

### General Demographics

In [ ]:
demographics = sql_to_df("./sql/demographics_general.sql")

In [ ]:
# TODO: correlate categorical demographics with sepsis target
# this includes: ethnicity, marital status, language, (pre-conditions?), (age groups?)

### Ethnicity of Hospital Patients

In [ ]:
asian_map = {
    "ASIAN": "Asian",
    "ASIAN - ASIAN INDIAN": "Asian",
    "ASIAN - CHINESE": "Asian",
    "ASIAN - KOREAN": "Asian",
    "ASIAN - SOUTH EAST ASIAN": "Asian",
}
black_map = {
    "BLACK": "Black",
    "BLACK/AFRICAN AMERICAN": "Black",
    "BLACK/AFRICAN": "Black",
    "BLACK/CAPE VERDEAN": "Black",
    "BLACK/CARIBBEAN ISLAND": "Black",
    "BLACK/HAITIAN": "Black",
}
white_map = {
    "WHITE": "White",
    "WHITE - BRAZILIAN": "White",
    "WHITE - EASTERN EUROPEAN": "White",
    "WHITE - OTHER EUROPEAN": "White",
    "WHITE - RUSSIAN": "White",
}
hispanic_map = {
    "HISPANIC": "Hispanic",
    "HISPANIC/LATINO - CENTRAL AMERICAN": "Hispanic",
    "HISPANIC/LATINO - COLOMBIAN": "Hispanic",
    "HISPANIC/LATINO - CUBAN": "Hispanic",
    "HISPANIC/LATINO - DOMINICAN": "Hispanic",
    "HISPANIC/LATINO - GUATEMALAN": "Hispanic",
    "HISPANIC/LATINO - HONDURAN": "Hispanic",
    "HISPANIC/LATINO - MEXICAN": "Hispanic",
    "HISPANIC/LATINO - PUERTO RICAN": "Hispanic",
    "HISPANIC/LATINO - SALVADORAN": "Hispanic",
    "HISPANIC OR LATINO": "Hispanic",
    "SOUTH AMERICAN": "Hispanic",
}
other_map = {
    "OTHER": "Other",
    "PORGTUGUESE": "Other",
    "MULTIPLE RAACE/ETHNICITY": "Other",
    "AMERICAN INDIAN/ALASKA NATIVE": "Other",
}
unknown_map = {
    "UNABLE TO OBTAIN": "Unknown",
    "UNKNOWN": "Unknown",
    "PATIENT DECLINED TO ANSWER": "Unknown",
}
all_map = asian_map | black_map | white_map | hispanic_map | other_map | unknown_map

In [ ]:
# run query
df = sql_to_df("./sql/race.sql")
# apply general ethnicity mapping
df["mapped_race"] = df["race"].map(all_map)

In [ ]:
ax = sns.histplot(data=df, x="mapped_race", hue="gender", multiple="dodge")
# add bar labels
for container in ax.containers:
    ax.bar_label(container, rotation=90)
# rotate x-tick labels
plt.xticks(rotation=90)

In [ ]:
from matplotlib.gridspec import GridSpec

asian = df[df["race"].isin(asian_map)]
black = df[df["race"].isin(black_map)]
white = df[df["race"].isin(white_map)]
hispanic = df[df["race"].isin(hispanic_map)]
other = df[df["race"].isin(other_map)]
unknown = df[df["race"].isin(unknown_map)]

fig = plt.figure(figsize=(20, 20))
gs = GridSpec(3, 3, figure=fig)

ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[1, 2])
ax5 = fig.add_subplot(gs[2, 0])
ax6 = fig.add_subplot(gs[2, 1])
ax7 = fig.add_subplot(gs[2, 2])

plot_pie_chart(df, "mapped_race", title="All Patients", ax=ax1)
plot_pie_chart(asian, title="Asian", ax=ax2, explode=0.05)
plot_pie_chart(black, title="Black", ax=ax3, explode=0.05)
plot_pie_chart(white, title="White", ax=ax4, explode=0.05)
plot_pie_chart(hispanic, title="Hispanic", ax=ax5, explode=0.05)
plot_pie_chart(other, title="Other", ax=ax6, explode=0.05)
plot_pie_chart(unknown, title="Unknown", ax=ax7, explode=0.05)

In [ ]:
# create color palette
palette = dict(zip(["F", "M"], sns.color_palette()))
# plot histograms
g = sns.displot(
    df,
    x="race",
    col="sepsis",
    hue="gender",
    facet_kws=dict(margin_titles=True),
    binwidth=2,
    height=6,
    kde=True,
    palette=palette,
)
g.set_titles(col_template="Sepsis={col_name}")
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle("Ethnicity", fontsize=16)
# rotate x-tick labels
for ax in g.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

#### Demographics in ICU

In [ ]:

# run icu query
dfICU = sql_to_df("./sql/race_ICU.sql")
dfICU["mapped_race"] = df["race"].map(all_map)

In [ ]:

ax = sns.histplot(data=dfICU, x="mapped_race", hue="gender", multiple="dodge")
#add bar labels
for container in ax.containers:
    ax.bar_label(container, rotation=90)
# rotate x-tick labels
plt.xticks(rotation=90)

In [ ]:
from matplotlib.gridspec import GridSpec

asian = dfICU[dfICU["race"].isin(asian_map)]
black = dfICU[dfICU["race"].isin(black_map)]
white = dfICU[dfICU["race"].isin(white_map)]
hispanic = dfICU[dfICU["race"].isin(hispanic_map)]
other = dfICU[dfICU["race"].isin(other_map)]
unknown = dfICU[dfICU["race"].isin(unknown_map)]

fig = plt.figure(figsize=(20, 20))
gs = GridSpec(3, 3, figure=fig)

ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[1, 2])
ax5 = fig.add_subplot(gs[2, 0])
ax6 = fig.add_subplot(gs[2, 1])
ax7 = fig.add_subplot(gs[2, 2])

plot_pie_chart(dfICU, "mapped_race", title="ICU Patients", ax=ax1)
plot_pie_chart(asian, title="Asian", ax=ax2, explode=0.05)
plot_pie_chart(black, title="Black", ax=ax3, explode=0.05)
plot_pie_chart(white, title="White", ax=ax4, explode=0.05)
plot_pie_chart(hispanic, title="Hispanic", ax=ax5, explode=0.05)
plot_pie_chart(other, title="Other", ax=ax6, explode=0.05)
plot_pie_chart(unknown, title="Unknown", ax=ax7, explode=0.05)

In [ ]:
# create color palette
palette = dict(zip(["F", "M"], sns.color_palette()))
# plot histograms
g = sns.displot(
    dfICU,
    x="race",
    col="sepsis",
    hue="gender",
    facet_kws=dict(margin_titles=True),
    binwidth=2,
    height=6,
    kde=True,
    palette=palette,
)
g.set_titles(col_template="Sepsis={col_name}")
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle("Ethnicity on ICU", fontsize=16)
# rotate x-tick labels
for ax in g.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

### Time to Infection - How long does it take for a patient to get infected after admission to the ICU?

In [ ]:
tti = sql_to_df("./sql/time_to_infection.sql")

In [ ]:
graph = sns.boxplot(x=tti["time_to_infection"])
# set x-axis scale to log (symmetric log scale to include negative values)
graph.set_xscale("symlog")
graph.tick_params(axis="x", which="minor", length=1)
# add vertical lines for ICU admission and ICU stay length at 8, 24 and 72 hours
graph.axvline(0, color="orange", linestyle="--")
graph.axvline(8, color="green", linestyle="--")
graph.axvline(24, color="green", linestyle="--")
graph.axvline(72, color="green", linestyle="--")
graph.text(
    0.05, 0.48, "ICU Admission", va="bottom", ha="left", color="orange", rotation=90
)
graph.text(
    8.5, 0.48, "8 hour ICU stay", va="bottom", ha="left", color="green", rotation=90
)
graph.text(
    25.5, 0.48, "24 hour ICU stay", va="bottom", ha="left", color="green", rotation=90
)
graph.text(
    74.5, 0.48, "72 hour ICU stay", va="bottom", ha="left", color="green", rotation=90
)
# add text box with summary statistics
graph.text(
    -35,
    0.45,
    tti["time_to_infection"].describe().round(3).to_string(),
    va="bottom",
    color="black",
    bbox=dict(facecolor="white", alpha=0.5),
)
plt.title("Sepsis Onset Time for ICU Patients")
plt.xlabel("Time to Infection (hours, log scale)")
plt.show()

In [ ]:
sns.histplot(data=tti, x="time_to_infection", stat="percent", cumulative=True, kde=True)
sns.rugplot(data=tti, x="time_to_infection", height=-0.02, clip_on=False, lw=1, alpha=0.5)
# set title 
plt.xlabel("Time to Infection (hours)")

#### Notes on Sepsis Onset Time

The MIMIC-IV authors note the following in their sepsis3 concept:
> As many variables used in SOFA are only collected in the ICU, this query can only define sepsis-3 onset within the ICU.
 
This somewhat explains the distribution of sepsis-3 onset times in the figure above. The distribution of sepsis-3 onset times is heavily skewed towards the point of admission to the ICU. Patients that may already have sepsis upon admission to the ICU will only be identified as having sepsis using the sepsis-3 criteria once the SOFA relevant variables have been collected. This is usually within the first few hours of admission to the ICU. It appears that the majority of patients that develop sepsis in the ICU do so within the first 24 hours of admission. Some patients have a recorded onset time that is well before their admission to the ICU. This is likely due to the fact that for some patients SOFA score variables may have been collected in the ED or the ward. This is the case for 39 patients in total. 31408 patients experience their sepsis onset in the first 24 hours of their icu stay. The mean onset time is 6.9h.

### Blood Gas Analysis - First 24 Hours in the ICU



In [ ]:
bg_first_24h = sql_to_df("./sql/first_day_bg.sql")

In [ ]:
# filter for columns with max/min values
bg_first_24h_min_max = bg_first_24h.filter(regex="(_max|_min|sepsis)$")
# plot correlation matrix
plot_corr_matrix(
    bg_first_24h_min_max,
    title="Blood Gas Analysis Correlation Matrix - Max/Min [24h ICU]",
    figsize=(25, 20),
    linewidth=0,
    cbar=True,
    cbar_kws={"shrink": 0.8},
)
# filter for columns with mean/std values
bg_first_24h_mean_std = bg_first_24h.filter(regex="(_mean|_std|sepsis)$")
# plot correlation matrix
plot_corr_matrix(
    bg_first_24h_mean_std,
    title="Blood Gas Analysis Correlation Matrix - Mean/Std [24h ICU]",
    figsize=(25, 20),
    linewidth=0,
    cbar=True,
    cbar_kws={"shrink": 0.8},
)

### Pre Septic Blood Gas Analysis - 8 Hours before Sepsis Onset

For patients without sepsis, the measurement interval is from ICU admission time to ICU admission time +8 hours.
This may or may not be a valid approach.

In [ ]:
# get blood gas analysis data for different time windows before sepsis onset
bg_pre_septic_2h = sql_to_df("./sql/pre_septic_bg.sql", {"window_size_h": 2})
bg_pre_septic_4h = sql_to_df("./sql/pre_septic_bg.sql", {"window_size_h": 4})
bg_pre_septic_6h = sql_to_df("./sql/pre_septic_bg.sql", {"window_size_h": 6})
bg_pre_septic_8h = sql_to_df("./sql/pre_septic_bg.sql", {"window_size_h": 8})

In [ ]:
# filter for columns with max/min values
bg_pre_septic_8h_min_max = bg_pre_septic_8h.filter(regex="(_max|_min|sepsis)$")
# plot correlation matrix
plot_corr_matrix(
    bg_pre_septic_8h_min_max,
    title="Blood Gas Analysis Correlation Matrix - Min/Max [8h Pre-Septic]",
    figsize=(25, 20),
    linewidth=0,
    cbar=True,
)
# filter for columns with mean/std values
bg_pre_septic_8h_mean_std = bg_pre_septic_8h.filter(regex="(_mean|_std|sepsis)$")
# plot correlation matrix
plot_corr_matrix(
    bg_pre_septic_8h_mean_std,
    title="Blood Gas Analysis Correlation Matrix - Mean/Std [8h Pre-Septic]",
    figsize=(25, 20),
    linewidth=0,
    cbar=True,
)

In [ ]:
# calculate correlation matrices
bg_corr_2h = bg_pre_septic_2h.corr().iloc[-1:]
bg_corr_4h = bg_pre_septic_4h.corr().iloc[-1:]
bg_corr_6h = bg_pre_septic_6h.corr().iloc[-1:]
bg_corr_8h = bg_pre_septic_8h.corr().iloc[-1:]

# add a column to record the time before sepsis onset
bg_corr_2h["h_before_onset"] = -2
bg_corr_4h["h_before_onset"] = -4
bg_corr_6h["h_before_onset"] = -6
bg_corr_8h["h_before_onset"] = -8

concat = pd.concat([bg_corr_2h, bg_corr_4h, bg_corr_6h, bg_corr_8h])
melted = concat.melt(id_vars=["sepsis", "h_before_onset"], value_vars=concat.columns[:-2])

grid = sns.relplot(
    data=melted,
    y="value",
    x="h_before_onset",
    col="variable",
    hue="sepsis",
    kind="line",
    col_wrap=11,
    height=2.5,
    facet_kws=dict(margin_titles=True),
    marker="o",
)
grid.set_titles(col_template="{col_name}")
grid.fig.tight_layout()
grid.fig.suptitle("Correlation between Blood Gas Components and Sepsis Onset Time", y=1.02, fontsize=16)

# draw horizontal line at y=0
for ax in grid.axes.flatten():
    ax.axhline(0, color="green", linestyle="--")

### Vital Signs - 24 hours after ICU admission

Technically its -6 to +24 hours after ICU admission. The first 6 hours are recorded because the first 6 hours are used to calculate the SOFA score.

In [ ]:
vs_first_24h = sql_to_df("./sql/first_day_vitalsign.sql")

In [ ]:
# filter for columns with max/min values
vs_first_24h_min_max = vs_first_24h.filter(regex="(_max|_min|sepsis|height|age)$")
# plot correlation matrix
plot_corr_matrix(
    vs_first_24h_min_max,
    title="Vital Signs Correlation Matrix - Min/Max [24h ICU]",
    figsize=(15, 10),
    linewidth=0,
)
# filter for columns with mean/std values
vs_first_24h_mean_std = vs_first_24h.filter(regex="(_mean|_std|sepsis|height|age)$")
# plot correlation matrix
plot_corr_matrix(
    vs_first_24h_mean_std,
    title="Vital Signs Correlation Matrix - Mean/Std [24h ICU]",
    figsize=(15, 10),
    linewidth=0,
)

### Vital Signs - 2/4/6/8 hours before Sepsis Onset

In [ ]:
vs_pre_septic_2h = sql_to_df("./sql/pre_septic_vitalsign.sql", {"window_size_h": 2})
vs_pre_septic_4h = sql_to_df("./sql/pre_septic_vitalsign.sql", {"window_size_h": 4})
vs_pre_septic_6h = sql_to_df("./sql/pre_septic_vitalsign.sql", {"window_size_h": 6})
vs_pre_septic_8h = sql_to_df("./sql/pre_septic_vitalsign.sql", {"window_size_h": 8})

In [ ]:
vs_pre_septic_8h_min_max = vs_pre_septic_8h.filter(regex="(_max|_min|sepsis|height|age)$")
# plot correlation matrices side by side
plot_corr_matrix(
    vs_pre_septic_8h_min_max,
    title="Vital Signs Correlation Matrix - Max/Min [8h Pre-Septic]",
    figsize=(15, 10),
    linewidth=0,
)
# filter for columns with mean/std values
vs_pre_septic_8h_mean_std = vs_pre_septic_8h.filter(regex="(_mean|_std|sepsis|height|age)$")
# plot correlation matrix
plot_corr_matrix(
    vs_pre_septic_8h_mean_std,
    title="Vital Signs Correlation Matrix - Mean/Std [8h Pre-Septic]",
    figsize=(15, 10),
    linewidth=0,
)

In [ ]:
plot_corr_matrix_diff(
    vs_first_24h_min_max,
    vs_pre_septic_8h_min_max,
    figsize=(15, 10),
    title="Vital Sign Correlation Difference - Min/Max [24h ICU - 8h Pre-Septic]",
)
plot_corr_matrix_diff(
    vs_first_24h_mean_std,
    vs_pre_septic_8h_mean_std,
    figsize=(15, 10),
    title="Vital Sign Correlation Difference - Mean/Std [24h ICU - 8h Pre-Septic]",
)

In [ ]:
# create a dataframe that records the correlation between each vital sign and sepsis
# use only the last row of the correlation matrix
vs_corr_2h = vs_pre_septic_2h.corr().iloc[-1:]
vs_corr_4h = vs_pre_septic_4h.corr().iloc[-1:]
vs_corr_6h = vs_pre_septic_6h.corr().iloc[-1:]
vs_corr_8h = vs_pre_septic_8h.corr().iloc[-1:]

# add a column to record the time before sepsis onset
vs_corr_2h["h_before_onset"] = -2
vs_corr_4h["h_before_onset"] = -4
vs_corr_6h["h_before_onset"] = -6
vs_corr_8h["h_before_onset"] = -8

# combine the dataframes into one, row-wise
concat = pd.concat([vs_corr_2h, vs_corr_4h, vs_corr_6h, vs_corr_8h])
melted = concat.melt(
    id_vars=["sepsis", "h_before_onset"], value_vars=concat.columns[:-2]
)

# plot as facetgrid
grid = sns.relplot(
    data=melted,
    y="value",
    x="h_before_onset",
    col="variable",
    hue="sepsis",
    kind="line",
    col_wrap=10,
    height=2.5,
    marker="o",
)
grid.set_titles(col_template="{col_name}")
grid.fig.suptitle(
    "Correlation between Blood Gas Components and Sepsis Onset Time",
    y=1.02,
    fontsize=16,
)
grid.fig.tight_layout()

# draw horizontal line at y=0
for ax in grid.axes.flatten():
    ax.axhline(0, color="green", linestyle="--")

### Lab Results

In [ ]:
lr_first_24h = sql_to_df("./sql/first_day_lab.sql")

In [ ]:
# filter for columns with max/min values
lr_first_24h_min_max = lr_first_24h.filter(regex="(_max|_min|sepsis)$")
# plot correlation matrix
plot_corr_matrix(
    lr_first_24h_min_max,
    title="Lab Results Correlation Matrix - Min/Max [24h ICU]",
    figsize=(35, 30),
    linewidth=0,
)
# filter for columns with mean/std values
lr_first_24h_mean_std = lr_first_24h.filter(regex="(_mean|_std|sepsis)$")
# plot correlation matrix
plot_corr_matrix(
    lr_first_24h_mean_std,
    title="Lab Results Correlation Matrix - Mean/Std [24h ICU]",
    figsize=(35, 30),
    linewidth=0,
)

In [ ]:
lr_pre_septic_2h = sql_to_df("./sql/pre_septic_lab.sql", {"window_size_h": 2})
lr_pre_septic_4h = sql_to_df("./sql/pre_septic_lab.sql", {"window_size_h": 4})
lr_pre_septic_6h = sql_to_df("./sql/pre_septic_lab.sql", {"window_size_h": 6})
lr_pre_septic_8h = sql_to_df("./sql/pre_septic_lab.sql", {"window_size_h": 8})

In [ ]:
lr_pre_septic_8h_min_max = lr_pre_septic_8h.filter(regex="(_max|_min|sepsis)$")

plot_corr_matrix(
    lr_pre_septic_8h_min_max,
    title="Lab Results Correlation Matrix - Min/Max [8h Pre-Septic]",
    figsize=(35, 30),
    linewidth=0,
)

lr_pre_septic_8h_mean_std = lr_pre_septic_8h.filter(regex="(_mean|_std|sepsis)$")

plot_corr_matrix(
    lr_pre_septic_8h_mean_std,
    title="Lab Results Correlation Matrix - Mean/Std [8h Pre-Septic]",
    figsize=(35, 30),
    linewidth=0,
)

In [ ]:
plot_corr_matrix_diff(
    lr_first_24h_min_max,
    lr_pre_septic_8h_min_max,
    figsize=(35, 30),
    title="Lab Results Correlation Difference - Min/Max [24h ICU - 8h Pre-Septic]",
)
plot_corr_matrix_diff(
    lr_first_24h_mean_std,
    lr_pre_septic_8h_mean_std,
    figsize=(35, 30),
    title="Lab Results Correlation Difference - Mean/Std [24h ICU - 8h Pre-Septic]",
)

lr_pre_septic_6h_min_max = lr_pre_septic_6h.filter(regex="(_max|_min|sepsis)$")
plot_corr_matrix_diff(
    lr_pre_septic_8h_min_max,
    lr_pre_septic_6h_min_max,
    figsize=(35, 30),
    title="Lab Results Correlation Difference - Min/Max [8h Pre-Septic - 6h Pre-Septic]",
)

In [ ]:
# create a dataframe that records the correlation between each vital sign and sepsis
# use only the last row of the correlation matrix
lr_corr_2h = lr_pre_septic_2h.corr().iloc[-1:]
lr_corr_4h = lr_pre_septic_4h.corr().iloc[-1:]
lr_corr_6h = lr_pre_septic_6h.corr().iloc[-1:]
lr_corr_8h = lr_pre_septic_8h.corr().iloc[-1:]

# add a column to record the time before sepsis onset
lr_corr_2h["h_before_onset"] = -2
lr_corr_4h["h_before_onset"] = -4
lr_corr_6h["h_before_onset"] = -6
lr_corr_8h["h_before_onset"] = -8

# combine the dataframes into one, row-wise
concat = pd.concat([lr_corr_2h, lr_corr_4h, lr_corr_6h, lr_corr_8h])
melted = concat.melt(
    id_vars=["sepsis", "h_before_onset"], value_vars=concat.columns[:-2]
)

# plot as facetgrid
grid = sns.relplot(
    data=melted,
    x="h_before_onset",
    y="value",
    col="variable",
    kind="line",
    col_wrap=10,
    height=2.5,
    marker="o",
)
grid.set_titles(col_template="{col_name}")
grid.fig.suptitle(
    "Correlation between Blood Gas Components and Sepsis Onset Time",
    y=1.02,
    fontsize=16,
)
grid.fig.tight_layout()

# draw horizontal line at y=0
for ax in grid.axes.flatten():
    ax.axhline(0, color="green", linestyle="--")

### Dialysis and Urine Output

In [ ]:
df_dia = sql_to_df("./sql/dialysis_urine_corr.sql")

In [ ]:
plot_corr_matrix(
    df_dia,
    "Dialysis and Urine Output - Correlation Matrix",
    figsize=(10, 5),
    linewidth=0,
)

### Glasgow Coma Scale (GCS)

The Glasgow Coma Scale (GCS) is a neurological scale which aims to give a reliable and objective way of recording the conscious state of a person for initial as well as subsequent assessment. A patient is assessed against the criteria of the scale, and the resulting points give a patient score between 3 (indicating deep unconsciousness) and either 14 (original scale) or 15 (more widely used modified or revised scale). The coma scale has three parameters: eye response (4), verbal response (5) and motor response (6) which are summed up to give the final score. The three values separately as well as their sum are considered in the evaluation of the patient's condition.

The MIMIC-IV concept table `first_day_gcs` contains the first day GCS score for patients in the ICU. Patients that are sedated are assigned a GCS score of 15. The authors of the concept recommend to use the last recorded GCS score before sedation for sedated patients (compare to SAPS II score). SAPS II is a severity score for ICU patients that is calculated based on the worst values for 12 physiological variables during the first 24 hours after admission. The GCS score is one of the 12 variables. Patients that receive ventilation through a tracheal tube are marked in the `gcs_unable` column.

In [ ]:
df = sql_to_df("./sql/pre_septic_gcs.sql")

In [ ]:
plot_corr_matrix(df, title="Correlation Matrix - Pre Septic Glasgow Coma Scale", figsize=(20, 10), linewidth=0)

#### Notes on the GCS Heatmap
The only positive correlation is between the `gcs_unable` column and the sepsis target variable. This is somewhat expected, as patients that are marked as unable are patients that receive ventilation through a tracheal tube. Ventilation is probably an indicator for the severity of the patients condition, as it is (probably) only necessary for more severe conditions. The other correlations will be negative, as the GCS score is a severity score. The lower the score, the more severe the condition. The gcs verbal subscore has the highest correlation between the GCS subscores and the sepsis target variable. 

Note that the query is skewed by design, since sedated patients receive a GCS of 15. This should be seen as a limitation of this specific query. The heatmap is still useful to get a general idea of the correlations between the GCS subscores and the sepsis target variable.

In [ ]:
# show distribution of gcs values
sns.displot(df, x="gcs_total", kind="kde", hue="sepsis", fill=True)

#### Notes on the Distribution of the GCS Score
The distribution of the GCS score is skewed to the right. The majority of patients have a GCS score of 15, which is the maximum score. We can observe, that sepsis patients have a higher density of lower GCS scores than non-sepsis patients. The distribution itself follow a similar pattern between sepsis/non-sepsis patients. This leads to the assumption, that the distribution of the GCS score may not be a good indicator for sepsis.

### Ventilation

In [ ]:
df_ventilation = sql_to_df("./sql/ventilation_corr.sql")

In [ ]:
plot_corr_matrix(df_ventilation, title="Correlation Matrix - Ventilation", method="pearson")

#### Notes on the Ventilation Heatmap
The GCS Heatmap contains a positive correlation between the `gcs_unable` column and the sepsis target variable. This heatmap shows a similar relation between the `non_invasive_vent` column aswell as the `invasive_vent` column and the sepsis target variable. This further corroborates the theory, that the `gcs_unable` column is actually just a correlation factor between ventilation and sepsis and has nothing to do with the actual GCS score.

We should note the following limitations for the query used to generate this heatmap:
- The query includes the whole ICU stay of the patient. This means that the patient may have been ventilated at some point during their ICU stay, but not necessarily during the first 24 hours.
- As the query contains the whole ICU stay, the patient may have been non-invasively ventilated at some point and then invasively ventilated at a later point. 
- The sepsis onset time is not considered in the query. This means that the query does not consider the ventilation status of the patient at the time of sepsis onset. This may be a useful insight to have, as it may be an indicator for sepsis onset.

### Patient Age Distribution

In [ ]:
all_ages = sql_to_df("./sql/all_ages.sql")

In [ ]:
plt.plot(all_ages[0], all_ages[1])
plt.title('Age of all patients')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

In [ ]:
icu_ages = sql_to_df("./sql/all_icu_ages.sql")

In [ ]:
plt.plot(icu_ages[0], icu_ages[1])
plt.title('Age of all ICU patients')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()